#### Data 
02. binary_products 
    - crawled html pages per each company 
03. textbinary_products 
    - markdown text pages per each company  
04. product details extraction 
    - json files per each company
        - ```json
            {
                filename.md : {
                    "product_name": "product_name",
                    "details": ["detail1", "detail2", "detail3"]
                }
            }
        ```
05. classified products 
    - json files per each company
        - ```json
            {
                filename.md : {
                    "product_name": "product_name",
                    "category": "category",
                    "details": ["detail1", "detail2", "detail3"]
                }
            }
        ``` 

In [74]:
import os
from dotenv import load_dotenv
import pandas as pd  # Import pandas to fix the error
from langchain.chat_models import ChatOpenAI

# Loading API KEY from environment
load_dotenv()
OpenAI_key = os.getenv("OPENAI_API_KEY") 
Huggingface_key = os.getenv("HUGGINGFACE_API_KEY")

chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

/var/folders/7_/lzvh2hfd7nbfj2n6q9k0hb980000gn/T/ipykernel_43739/3967557882.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model="gpt-4o", temperature=0)


#### Classify products 

In [1]:
import os 

detail_data_dir  = "data/2_detailed_json_data/"
detail_json_dirs = [] 

# classification_data_dir = "data/3_classified_json_data/" 
classification_data_dirs = []

classification_prompt_file = "data/prompts/classification_prompt.txt"
categories_file = "data/0_insurance_categories.txt"

# load detail json files, and create classified json file names 
for filename in os.listdir(detail_data_dir):
    if filename.endswith('.json'):
        detail_data = detail_data_dir + filename
        detail_json_dirs.append(detail_data)
        company_name = filename.split('_')[0]

        classification_output_data = company_name + "_product_classified.json"
        classification_data_dirs.append(classification_output_data)

print(detail_json_dirs) # input data names 
print(classification_data_dirs) # output data names 

['data/2_detailed_json_data/axa_product_details.json', 'data/2_detailed_json_data/arag_product_details.json']
['axa_product_classified.json', 'arag_product_classified.json']


In [32]:
print(load_prompt_from_file(classification_prompt_file))

Der Name des Produkts ist: {product_name}
Details des Produkts:
{details}

Kategorisiere dieses Produkt basierend auf der folgenden Liste von Kategorien:
{categories}

Wenn das Produkt zu keiner der Kategorien passt, antworte mit 'Keine Übereinstimmung'.
Ansonsten antworte nur mit dem Namen der passenden Kategorie.



In [33]:
def load_prompt_from_file(prompt_file):
    """Load the prompt content from a text file."""
    with open(prompt_file, 'r', encoding='utf-8') as file:
        return file.read() 
    
def load_categories_from_file(categories_file):
    """Load the list of categories from a text file."""
    with open(categories_file, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines() if line.strip()]

In [34]:
import json
from langchain.prompts import PromptTemplate
import os

def load_categories_from_file(categories_file):
    """Load the list of categories from a text file."""
    with open(categories_file, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines() if line.strip()]

def classify_products(input_file: str, output_file: str, prompt_file: str, categories_file: str, chat_model):
    # Load product details
    with open(input_file, 'r', encoding='utf-8') as file:
        product_details = json.load(file)

    # Load the prompt and categories
    prompt_template = load_prompt_from_file(prompt_file)
    categories = load_categories_from_file(categories_file)

    classified_products = {}

    # Create a LangChain prompt
    prompt = PromptTemplate(
        input_variables=["product_name", "details", "categories"],
        template=prompt_template
    )

    # Iterate over each product in the JSON
    for product_file, product_data in product_details.items():
        product_name = product_data.get("product_name", "").strip()
        details = "\n".join(product_data.get("details", []))

        if not product_name:  # Skip products with no name
            continue

        # Format the prompt
        formatted_prompt = prompt.format(
            product_name=product_name,
            details=details,
            categories="\n".join(categories)
        )

        # Use the chat model to classify the product
        try:
            response = chat_model.predict(formatted_prompt).strip()
            if response in categories:  # Check if the response matches a category
                classified_products[product_file] = {
                    "product_name": product_name,
                    "category": response,
                    "details": product_data["details"]
                }
            else:
                # print(f"Keine Übereinstimmung für: {product_name}")
                print(f"no match for: {product_name}")

        except Exception as e:
            # print(f"Fehler bei der Verarbeitung von {product_name}: {e}")
            print(f"error for {product_name}: {e}")


    # Create the directory if it doesn't exist
    output_dir = 'data/2_detailed_json_data'
    os.makedirs(output_dir, exist_ok=True)
    output_file_path = os.path.join(output_dir, output_file)

    # Save the classified products to the new JSON file
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(classified_products, outfile, ensure_ascii=False, indent=4)

    # print(f"Klassifizierte Produkte gespeichert in {output_file}")
    print(f"Classified products saved in {output_file}") 
    return classified_products

for input_file, output_file in zip(detail_json_dirs, classification_data_dirs):
    print(f"Processing {input_file} and saving to {output_file}")
    _ = classify_products(input_file, output_file, classification_prompt_file, categories_file, chat_model)

NameError: name 'detail_json_dirs' is not defined

## Comparison

In [75]:
import os
from dotenv import load_dotenv
import pandas as pd  # Import pandas to fix the error
from langchain.chat_models import ChatOpenAI

# Loading API KEY from environment
load_dotenv()
OpenAI_key = os.getenv("OPENAI_API_KEY") 
Huggingface_key = os.getenv("HUGGINGFACE_API_KEY")

chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

In [37]:
def load_prompt_from_file(prompt_file):
    """Load the prompt content from a text file."""
    with open(prompt_file, 'r', encoding='utf-8') as file:
        return file.read() 
    
def load_categories_from_file(categories_file):
    """Load the list of categories from a text file."""
    with open(categories_file, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines() if line.strip()]

In [38]:
import os 

classification_data_dir = "data/3_classified_json_data/" 
classification_data_dirs = []

comparison_prompt_file = "data/prompts/comparison_prompt.txt"
categories_file = "data/0_insurance_categories.txt"

# load detail json files, and create classified json file names 
for filename in os.listdir(classification_data_dir):
    if filename.endswith('.json'):
        detail_data = classification_data_dir + filename
        classification_data_dirs.append(detail_data)

print(classification_data_dirs)


['data/3_classified_json_data/arag_product_classified.json', 'data/3_classified_json_data/axa_product_classified.json']


In [ ]:
import os
import json
from langchain.prompts import PromptTemplate

def create_product_comparisons(axa_file, arag_file, output_file, prompt_file, categories_file, chat_model):
    # Load classified products
    with open(axa_file, 'r', encoding='utf-8') as file:
        axa_products = json.load(file)

    with open(arag_file, 'r', encoding='utf-8') as file:
        arag_products = json.load(file)

    # Load the prompt and categories
    prompt_template = \
    """Erstellen Sie eine Vergleichstabelle im Markdown-Format für das Produkt '{category}'
    von zwei Versicherungsunternehmen, AXA und ARAG. Die Vergleichstabelle sollte die
    Produktdetails in drei Spalten enthalten: Eigenschaft, AXA, und ARAG.

    Details von AXA:
    {axa_details}

    Details von ARAG:
    {arag_details}

    Vergleichstabelle:
    """

    categories = load_categories_from_file(categories_file)

    # Create a LangChain prompt
    prompt = PromptTemplate(
        input_variables=["category", "axa_details", "arag_details"],
        template=prompt_template
    )

    # Create the directory if it doesn't exist
    output_dir = 'data/4_comparison_data/'
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through all categories
    for category in categories:
        if category != "Reiseversicherung":
            continue
        # Find products matching the category
        axa_product = None
        arag_product = None

        for product_data in axa_products.values():
            if product_data["category"] == category:
                axa_product = product_data
                break

        for product_data in arag_products.values():
            if product_data["category"] == category:
                arag_product = product_data
                break

        # Skip if one of the companies does not have the product
        if not axa_product or not arag_product:
            print(f"No comparison for category: {category}")
            continue


        # Format the prompt
        formatted_prompt = prompt.format(
            category=category,
            axa_details="\n".join(axa_product["details"]),
            arag_details="\n".join(arag_product["details"]),
        )

        # Generate the comparison table
        try:
            response = chat_model.predict(formatted_prompt).strip()
            
            output_file_path = os.path.join(output_dir, f"{category}_{output_file}")
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.write(response)

            print(f"Comparison table saved for {category} in {output_file}")

        except Exception as e:
            print(f"Error generating comparison for {category}: {e}")

for i, classification_data in enumerate(classification_data_dirs):
    # Ensure the length of classification_data_dirs is sufficient to avoid IndexError
    if i + 1 < len(classification_data_dirs):
        company_1 = classification_data
        company_2 = classification_data_dirs[i + 1]
    else:
        print("No more companies to compare")
        continue
    company_1 = classification_data
    company_2 = classification_data_dirs[i+1]

    company_1_name = company_1.split('/')[-1].split('_')[0]
    company_2_name = company_2.split('/')[-1].split('_')[0]

    output_file = company_1_name + "_" + company_2_name + "_comparison.md"

    # Generate comparisons for all products
    create_product_comparisons(company_1, company_2, output_file, comparison_prompt_file, categories_file, chat_model)

========================================================================================================================================================================================================

## Convert the already prepared classified data into CSV for Kedro implementation !!

In [76]:
import os
from dotenv import load_dotenv
import pandas as pd  # Import pandas to fix the error
from langchain.chat_models import ChatOpenAI

# Loading API KEY from environment
load_dotenv()
OpenAI_key = os.getenv("OPENAI_API_KEY") 
Huggingface_key = os.getenv("HUGGINGFACE_API_KEY")

chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

In [77]:
import json
import os
import pandas as pd

classification_json_data_dir = "data/3_classified_json_data/"
classification_csv_data_dir = "data/3_classified_csv_data/"

# Ensure the CSV directory exists
os.makedirs(classification_csv_data_dir, exist_ok=True)

# Initialize an empty list to store all products
all_products = []

# Iterate over each JSON file in the directory
for product_file in os.listdir(classification_json_data_dir):
    if product_file.endswith(".json"):
        with open(os.path.join(classification_json_data_dir, product_file), 'r', encoding='utf-8') as file:
            product_data = json.load(file)

            company_name = product_file.split("_")[0]  # Change to match the company name from file names 
            for product in product_data.values(): 
                product_name = product["product_name"]
                category = product["category"]
                details = "\n".join(product["details"])
            
                all_products.append({
                    "company_name": company_name,
                    "product_name": product_name,
                    "category": category,
                    "details": details
                })

# Create a single DataFrame for all products
all_products_df = pd.DataFrame(all_products)
all_products_df.to_csv(os.path.join(classification_csv_data_dir, "classified_data.csv"), index=False)

In [67]:
all_products_df

,company_name,product_name,category,details
0,arag,Haftpflichtversicherungen,Haftpflichtversicherung,Privathaftpflicht: Schutz vor finanziellen Aus...
1,arag,Betriebshaftpflichtversicherung,Haftpflichtversicherung,Schutz für jede Branche\nVersicherungsschutz b...
2,arag,Mietrechtsschutz,Rechtsschutzversicherung,Schützt Interessen als Mieter durch alle Insta...
3,arag,Top-Schutzbrief,Reiseversicherung,Bietet Unterstützung in Notsituationen rund um...
4,arag,Zahnzusatzversicherung,Krankenversicherung,4 Tarife zur Wahl für den passenden Schutz\nAl...
...,...,...,...,...
113,axa,Symptom-Check,Krankenversicherung,Zuverlässig online Symptome checken\nIn wenige...
114,axa,Mitarbeiterabsicherung - Corporate Employee Be...,Krankenversicherung,Unterstützung bei der Mitarbeiterbindung und -...
115,axa,Lieferwagen-Versicherung,Haftpflichtversicherung,"Extra-Sicherheit für Ihren Lkw bis zu 3,5 t zu..."
116,axa,Kfz-Flottenversicherung,Haftpflichtversicherung,"Empfohlen für Unternehmen, die mehr als 10 Kra..."


In [78]:
def load_prompt_from_file(prompt_file):
    """Load the prompt content from a text file."""
    with open(prompt_file, 'r', encoding='utf-8') as file:
        return file.read() 
    
def load_categories_from_file(categories_file):
    """Load the list of categories from a text file."""
    with open(categories_file, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file.readlines() if line.strip()]
    
categories_file = "data/0_insurance_categories.txt"
comparison_prompt_file = 'data/prompts/comparison_prompt.txt'

In [79]:
# TODO : Put the predefined categories into a pydantic model 
from langchain.prompts import PromptTemplate 

output_dir = 'data/4_comparison_csv_data'
os.makedirs(output_dir, exist_ok=True)


prompt_template = load_prompt_from_file(comparison_prompt_file)
prompt = PromptTemplate(
    input_variables=["category", "company_1", "company_2", "company_1_details", "company_2_details"],
    template=prompt_template
)

company_names = all_products_df['company_name'].unique()
company_names_n = len(company_names)
predefined_category_list = load_categories_from_file(categories_file)

# Initialize a list to store the data for the DataFrame
comparison_data = []

# Iterate over all company names and get all rows for each pair of company names
for i in range(0, company_names_n):
    if i+1 < company_names_n:
        company_1 = company_names[i]
        company_2 = company_names[i+1]
    else:
        break

    # Get rows for company_1 and company_2
    company_1_rows = all_products_df[all_products_df['company_name'] == company_1]
    company_2_rows = all_products_df[all_products_df['company_name'] == company_2]

    # Find common categories
    common_categories = list(set(company_1_rows['category']).intersection(set(company_2_rows['category'])))

    for category in common_categories[:5]:
        print(category)
        if category not in predefined_category_list:
            break
        company_1_category_products = company_1_rows[company_1_rows['category'] == category]
        company_2_category_products = company_2_rows[company_2_rows['category'] == category]

        company_1_product_info_list = company_1_category_products.apply(lambda row: f"Product Name: {row['product_name']}\nDetails: {row['details']}", axis=1).to_list()
        company_2_product_info_list = company_2_category_products.apply(lambda row: f"Product Name: {row['product_name']}\nDetails: {row['details']}", axis=1).to_list()

        company_1_product_details = "\n".join(company_1_product_info_list)
        company_2_product_details = "\n".join(company_2_product_info_list) 

        # Format the prompt
        formatted_prompt = prompt.format(
            company_1=company_1,
            company_2=company_2,
            category=category,
            company_1_details=company_1_product_details,
            company_2_details=company_2_product_details,
        )
 
        response = chat_model.predict(formatted_prompt).strip() 

        # Append the data to the list
        comparison_data.append({
            "company_1": company_1,
            "company_2": company_2,
            "category": category,
            "company_1_details": company_1_product_details,
            "company_2_details": company_2_product_details,
            "comparison": response,
            "prompt": prompt.format(
                company_1=company_1,
                company_2=company_2,
                category=category,
                company_1_details=company_1_product_details,
                company_2_details=company_2_product_details,
            )
        })

# Convert the list to a DataFrame
comparison_df = pd.DataFrame(comparison_data)

Gewerbeversicherung


/var/folders/7_/lzvh2hfd7nbfj2n6q9k0hb980000gn/T/ipykernel_43739/1618777344.py:58: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chat_model.predict(formatted_prompt).strip()


Betriebsunterbrechungsversicherung
Transportversicherung
Reiseversicherung
Krankenversicherung


In [80]:
comparison_df

,company_1,company_2,category,company_1_details,company_2_details,comparison,prompt
0,arag,axa,Gewerbeversicherung,Product Name: Betriebliche Krankenversicherung...,Product Name: Cyber-Versicherung\nDetails: Umf...,Below is a comparison table in Markdown format...,Create a comparison table in Markdown format f...
1,arag,axa,Betriebsunterbrechungsversicherung,Product Name: Ertragsausfallversicherung\nDeta...,Product Name: Betriebsschließungsversicherung\...,Here's a comparison table in Markdown format f...,Create a comparison table in Markdown format f...
2,arag,axa,Transportversicherung,Product Name: Werkverkehrsversicherung\nDetail...,Product Name: Autoinhaltsversicherung\nDetails...,Here's a comparison table in Markdown format f...,Create a comparison table in Markdown format f...
3,arag,axa,Reiseversicherung,Product Name: Top-Schutzbrief\nDetails: Bietet...,Product Name: Auslandskrankenversicherung\nDet...,Here's a comparison table in Markdown format f...,Create a comparison table in Markdown format f...
4,arag,axa,Krankenversicherung,Product Name: Zahnzusatzversicherung\nDetails:...,Product Name: Betriebliche Krankenversicherung...,Below is a comparison table in Markdown format...,Create a comparison table in Markdown format f...


In [81]:
# Save comparison_df to csv in data/4_comparison_csv_data folder
output_path = "data/4_comparison_csv_data/comparison_data.csv"
comparison_df.to_csv(output_path, index=False)

## Evaluation 

In [82]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables and initialize clients
load_dotenv()
OpenAI_key = os.getenv("OPENAI_API_KEY")  
Huggingface_key = os.getenv("HUGGINGFACE_API_KEY")
client = OpenAI()

In [83]:
import pandas as pd
comparison_df = pd.read_csv('data/4_comparison_csv_data/comparison_data.csv')

comparison_df['companies_info'] = comparison_df.apply(
    lambda row: f"""{row['company_1']} and {row['company_2']} comparison in {row['category']}: \n\n
    {row['company_1']} information : \n\n {row['company_1_details']}\n\n
    {row['company_2']} information : \n\n {row['company_2_details']}""", axis=1) 

In [84]:
comparison_df

In [86]:
from trulens.core import TruSession
from trulens.dashboard import run_dashboard

session = TruSession()
session.reset_database()

decorating <function ComparisonApp.compare at 0x1679bfba0>
adding method <class '__main__.ComparisonApp'> compare __main__


In [ ]:
golden_set = (
    comparison_df[['companies_info', 'comparison']]
    .rename(columns={'companies_info': 'query', 'comparison': 'expected_response'})
    .to_dict("records")
)

In [ ]:
from trulens.providers.huggingface import Huggingface
from trulens.providers.openai import OpenAI
from trulens.feedback import GroundTruthAgreement


hug_provider = Huggingface() # for groundedness_measure_with_nli 
provider = OpenAI(model_engine="gpt-4o")
ground_truth_collection = GroundTruthAgreement(golden_set, provider=provider)
gta = GroundTruthAgreement(golden_set, provider=provider)

for i in range(len(comparison_df)):
    similarity_score = gta.agreement_measure(
        prompt=comparison_df['companies_info'][i],
        response=comparison_df['comparison'][i]
    )[0]

    comprehensiveness_score = provider.comprehensiveness_with_cot_reasons( # https://www.trulens.org/cookbook/use_cases/summarization_eval/#write-feedback-functions 
        source=comparison_df['companies_info'][i], # The source that should support the statement ( comapnies information )
        summary=comparison_df['comparison'][i], # Statement ( comparison result ) 
    )[0]

    groundedness_score = provider.groundedness_measure_with_cot_reasons( # https://www.trulens.org/cookbook/use_cases/summarization_eval/#write-feedback-functions 
        source=comparison_df['companies_info'][i], # The source that should support the statement ( comapnies information )
        statement=comparison_df['comparison'][i], # Statement ( comparison result ) 
    )[0]

    print(f'comparison between {comparison_df["company_1"][i]} and {comparison_df["company_2"][i]} in {comparison_df["category"][i]}')
    print(f'similarity_score : {similarity_score}, comprehensiveness_score : {comprehensiveness_score}, groundedness_score : {groundedness_score}')